In [58]:
### @credit: gensim tutorial on LDA and ensembleLda
import gensim
from spacy.lang.en.stop_words import STOP_WORDS
def tokenize(sentence):
    return [token for token in sentence.split() if token not in STOP_WORDS]
import re
def clean_sentence(sentence):
    sentence = sentence.lower().strip()
    sentence = re.sub(r'[^a-z0-9\s]', '', sentence)
    return re.sub(r'\s{2,}', ' ', sentence)
from gensim.models.phrases import Phrases, Phraser
def build_phrases(sentences):
    phrases = Phrases(sentences,
                      min_count=1,
                      threshold=10,
                      progress_per=1000)
    return Phraser(phrases)
def sentence_to_bi_grams(phrases_model, sentence):
    return ' '.join(phrases_model[sentence])


In [63]:
#from itertools import pairwise

In [64]:
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.DEBUG)
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
for handler in logger.handlers:
    handler.setLevel(logging.DEBUG)

In [65]:
p = []
with open("phrases.txt") as f:
    for line in f:
        phrase = line.split()
        prefix = phrase[0]
        for x in phrase[1:]:
            p.append((prefix, x))
            prefix += "_" + x
p = dict(p)

In [66]:
p

{'effective': 'predictors',
 'high': 'probabilities',
 'like': 'hold',
 'maximum': 'likelihood',
 'space': 'model',
 'sub': 'linear',
 'tf': 'idf',
 'united': 'nations',
 'word': 'distributions',
 'decision': 'making',
 'previous': 'slide',
 'social': 'networks',
 'active': 'research',
 'active_research': 'area',
 'ad': 'hoc',
 'ad_hoc': 'information',
 'beta': 'values',
 'big': 'picture',
 'categorisation': 'results',
 'causal': 'topics',
 'cause': 'overfitting',
 'cheaper': 'hotels',
 'clicked': 'documents',
 'complete': 'understanding',
 'conditional': 'entropy',
 'cranfield': 'evaluation',
 'cranfield_evaluation': 'methodology',
 'deeper': 'natural',
 'detailed': 'understanding',
 'discovering': 'syntagmatic',
 'eats': 'occurs',
 'expected': 'count',
 'expensive': 'hotels',
 'fewer': 'parameters',
 'generally': 'preferred',
 'generation': 'process',
 'geometric': 'mean',
 'gold': 'standard',
 'good': 'hubs',
 'google': 'file',
 'google_file': 'system',
 'gradually': 'group',
 'grou

In [67]:
docs = []
with open("textretrieval.txt", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
with open("textanalytics.txt") as f:
    for line in f:
        docs.append(line.split('.'))
with open("noncs.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
with open("cs125.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
with open("bkgd.dat", encoding='utf-8') as f:
    for line in f:
        docs.append(line.split('.'))
print(len(docs))
docs = [[clean_sentence(s) for s in sentences] for sentences in docs]
docs = [[tokenize(s) for s in sentences] for sentences in docs]

phrased = []
for sentences in docs:
    ps = []
    for s in sentences:
        i = 0
        new_s = []
        while i < len(s) - 1:
            if (s[i+1] == p.get(s[i], False)):
                new_s.append(s[i] + "_" + s[i+1])
                i+=2
            else:
                new_s.append(s[i])
                i+=1
        nns =[]
        i = 0
        while i < len(new_s) - 1:
            if (new_s[i+1] == p.get(new_s[i], False)):
                nns.append(new_s[i] + "_" + new_s[i+1])
                i+=2
            else:
                nns.append(new_s[i])
                i+=1
        ps.append(nns)
    phrased.append(ps)
len(phrased)

160


160

In [68]:
phrased_doc = []
for doc in phrased:
    d = []
    for s in doc:
        d+=s
    phrased_doc.append(d)
phrased_doc

[['lecture',
  'natural_language',
  'picture',
  'step',
  'process',
  'text',
  'data',
  'text',
  'data',
  'computers',
  'understand',
  'natural_language',
  'extent',
  'order',
  'thats',
  'going',
  'natural_language_processing',
  'main',
  'technique',
  'processing',
  'natural_language',
  'obtain',
  'understanding',
  'second',
  'state',
  'art',
  'nlp',
  'stands',
  'finally',
  'going',
  'cover',
  'relation',
  'natural_language_processing',
  'nlp',
  'best',
  'way',
  'explain',
  'think',
  'text',
  'foreign',
  'order',
  'understand',
  'text',
  'basically',
  'computers',
  'facing',
  'right',
  'looking',
  'simple',
  'sentence',
  'like',
  'dog',
  'chasing',
  'dont',
  'problem',
  'imagine',
  'computer',
  'order',
  'understand',
  'know',
  'dogs',
  'noun',
  'chasing',
  'called',
  'lexical',
  'analysis',
  'need',
  'figure',
  'syntactic',
  'going',
  'figure',
  'example',
  'shows',
  'dog',
  'form',
  'wont',
  'dog',
  'structure

In [21]:
    # with open("miss.dat", "w") as f:
#     for doc in phrased_doc:
#         f.write(" ".join(doc)+"\n")

In [69]:
from gensim.models import LdaModel

In [70]:
from gensim.corpora import Dictionary
import random

dictionary = Dictionary(phrased_doc)
dictionary.filter_extremes(no_below=2, no_above=0.1)

corpus = [dictionary.doc2bow(doc) for doc in phrased_doc]
num_topics = 86*2
chunksize = 160 # how many documents to process at a time
passes = 20 # epochs
iterations = 100
eval_every = 10

# Make a index to word dictionary.
temp = dictionary[0] 
id2word = dictionary.id2token

model = LdaModel(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    alpha='auto',
    eta='auto',
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every
)


2021-12-04 17:45:05,628 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2021-12-04 17:45:05,712 : INFO : built Dictionary<11682 unique tokens: ['90', 'able', 'accuracy', 'accurately', 'achieve']...> from 160 documents (total 198277 corpus positions)
2021-12-04 17:45:05,713 : DEBUG : starting a new internal lifecycle event log for Dictionary
2021-12-04 17:45:05,713 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<11682 unique tokens: ['90', 'able', 'accuracy', 'accurately', 'achieve']...> from 160 documents (total 198277 corpus positions)", 'datetime': '2021-12-04T17:45:05.713112', 'gensim': '4.1.3.dev0', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-12-04 17:45:05,722 : INFO : discarding 6470 tokens: [('able', 85), ('achieve', 30), ('act', 18), ('actually', 134), ('add', 61), ('additional', 31), ('allow', 73), ('alright', 45), ('analysis', 35), ('applet', 1)].

2021-12-04 17:45:06,981 : INFO : topic diff=131.831635, rho=1.000000
2021-12-04 17:45:07,041 : DEBUG : bound: at document #0
2021-12-04 17:45:07,261 : INFO : -13.549 per-word bound, 11985.9 perplexity estimate based on a held-out corpus of 160 documents with 47442 words
2021-12-04 17:45:07,262 : INFO : PROGRESS: pass 1, at document #160/160
2021-12-04 17:45:07,262 : DEBUG : performing inference on a chunk of 160 documents
2021-12-04 17:45:07,388 : DEBUG : 159/160 documents converged within 100 iterations
2021-12-04 17:45:07,391 : INFO : optimized alpha [0.0056162067, 0.005709298, 0.00561625, 0.0056737405, 0.005688517, 0.0056594047, 0.005674508, 0.0055807955, 0.005687649, 0.005780828, 0.005649637, 0.005780631, 0.005616883, 0.005688676, 0.0056522423, 0.0056880685, 0.005650384, 0.005738016, 0.0055666105, 0.0056587583, 0.0057217353, 0.0057370467, 0.0060961666, 0.005652208, 0.0057497174, 0.005801159, 0.0056950077, 0.0055812714, 0.005825876, 0.005652288, 0.005982673, 0.005617079, 0.005693684

2021-12-04 17:45:07,974 : DEBUG : bound: at document #0
2021-12-04 17:45:08,145 : INFO : -9.887 per-word bound, 946.9 perplexity estimate based on a held-out corpus of 160 documents with 47442 words
2021-12-04 17:45:08,145 : INFO : PROGRESS: pass 3, at document #160/160
2021-12-04 17:45:08,146 : DEBUG : performing inference on a chunk of 160 documents
2021-12-04 17:45:08,231 : DEBUG : 160/160 documents converged within 100 iterations
2021-12-04 17:45:08,235 : INFO : optimized alpha [0.005442467, 0.005563914, 0.0054425076, 0.005530188, 0.0055103097, 0.0055501405, 0.0055309306, 0.005409222, 0.0055094957, 0.005596838, 0.0056415712, 0.0056313183, 0.0054431017, 0.005510461, 0.005476283, 0.0055098888, 0.00547454, 0.005659206, 0.0054292073, 0.0055160276, 0.0055414587, 0.005624376, 0.0060742097, 0.0054762512, 0.005669912, 0.0058230245, 0.0056178165, 0.005409668, 0.005743374, 0.0054763258, 0.005892663, 0.0054432857, 0.0056495597, 0.005512616, 0.005496478, 0.005550193, 0.0055296123, 0.0055307825

2021-12-04 17:45:08,973 : INFO : -8.645 per-word bound, 400.4 perplexity estimate based on a held-out corpus of 160 documents with 47442 words
2021-12-04 17:45:08,973 : INFO : PROGRESS: pass 5, at document #160/160
2021-12-04 17:45:08,974 : DEBUG : performing inference on a chunk of 160 documents
2021-12-04 17:45:09,060 : DEBUG : 160/160 documents converged within 100 iterations
2021-12-04 17:45:09,064 : INFO : optimized alpha [0.005305883, 0.005448898, 0.0053059217, 0.005416579, 0.0053703208, 0.0054629394, 0.005417307, 0.0052742916, 0.0053695487, 0.0054524476, 0.0056341817, 0.0055406312, 0.0053064865, 0.005370465, 0.0053380067, 0.0053699217, 0.0053363517, 0.005595758, 0.0053203353, 0.0054030563, 0.0053998944, 0.0055345506, 0.0060556554, 0.0053379764, 0.0056056636, 0.0058396505, 0.005555577, 0.0052747154, 0.0056767133, 0.005338048, 0.0058203954, 0.00530666, 0.005585826, 0.005427051, 0.005384448, 0.005462811, 0.0054433565, 0.00541728, 0.005589677, 0.0054019773, 0.0055102557, 0.005879230

2021-12-04 17:45:09,783 : INFO : PROGRESS: pass 7, at document #160/160
2021-12-04 17:45:09,783 : DEBUG : performing inference on a chunk of 160 documents
2021-12-04 17:45:09,858 : DEBUG : 160/160 documents converged within 100 iterations
2021-12-04 17:45:09,862 : INFO : optimized alpha [0.005191745, 0.0053522987, 0.005191782, 0.0053211376, 0.005253409, 0.005389202, 0.005321854, 0.005161501, 0.005252669, 0.005331953, 0.0056272955, 0.0054640174, 0.005192322, 0.005253547, 0.00522249, 0.0052530277, 0.005220906, 0.0055417186, 0.00522878, 0.0053081377, 0.005281698, 0.005458639, 0.0060392586, 0.0052224603, 0.005550946, 0.005853147, 0.0055025276, 0.005161907, 0.0055958405, 0.0052225287, 0.005759014, 0.0051924884, 0.0055315616, 0.0053547104, 0.005290302, 0.005388913, 0.005370407, 0.0053450335, 0.005535906, 0.00528369, 0.005459066, 0.005853842, 0.0056495788, 0.0052242256, 0.005223399, 0.0055249436, 0.0053192996, 0.0053227516, 0.0052223, 0.005402905, 0.005507482, 0.005161304, 0.0051923012, 0.005

...

2021-12-04 17:45:11,434 : DEBUG : 160/160 documents converged within 100 iterations
2021-12-04 17:45:11,437 : INFO : optimized alpha [0.005005472, 0.0051937033, 0.0050055063, 0.0051643983, 0.0050627463, 0.005267162, 0.005165092, 0.0049773636, 0.00506206, 0.005135628, 0.0056146123, 0.0053373515, 0.0050060092, 0.0050628753, 0.005034034, 0.005062393, 0.0050325636, 0.0054515013, 0.005078244, 0.005152233, 0.005089006, 0.0053330823, 0.006010646, 0.005034007, 0.0054596146, 0.0059059295, 0.0054139094, 0.0049777403, 0.005462283, 0.005034071, 0.0056568785, 0.0050061624, 0.005441007, 0.0052349465, 0.0051356293, 0.005266625, 0.0052496153, 0.0052254437, 0.0054461355, 0.0050908546, 0.005373497, 0.0058641434, 0.005553275, 0.005035647, 0.005034879, 0.005436294, 0.0051812534, 0.005166339, 0.005033858, 0.0053090444, 0.00541832, 0.0049771797, 0.005005989, 0.005005555, 0.0053084684, 0.005297324, 0.005496455, 0.0052234773, 0.00535464, 0.0050342064, 0.005136126, 0.005839649, 0.0052406397, 0.0052404357, 0.00

2021-12-04 17:45:12,213 : DEBUG : 160/160 documents converged within 100 iterations
2021-12-04 17:45:12,216 : INFO : optimized alpha [0.0049267174, 0.0051262975, 0.004926751, 0.005097764, 0.0049821865, 0.005214923, 0.0050984477, 0.004899488, 0.004981522, 0.0050527425, 0.0056086676, 0.0052831825, 0.004927238, 0.0049823117, 0.004954382, 0.0049818447, 0.004952958, 0.005412589, 0.0050141797, 0.005085944, 0.0050076116, 0.0052793683, 0.005997817, 0.0049543553, 0.0054202294, 0.005933361, 0.005375668, 0.0048998524, 0.005405219, 0.0049544177, 0.0056129475, 0.0049273865, 0.005401957, 0.0051836623, 0.005069851, 0.0052142865, 0.0051978948, 0.005174237, 0.005407412, 0.005009401, 0.0053365463, 0.0058732466, 0.005511797, 0.004955944, 0.0049552, 0.005398038, 0.00513117, 0.005099835, 0.004954211, 0.005273325, 0.0053798496, 0.0048993104, 0.004927219, 0.0049267984, 0.005255214, 0.0052444465, 0.0054567493, 0.005155218, 0.005300148, 0.004954549, 0.005070398, 0.0058107655, 0.0051894826, 0.0051716017, 0.0054

2021-12-04 17:45:12,974 : INFO : optimized alpha [0.004854954, 0.005064692, 0.004854986, 0.005036857, 0.0049088043, 0.0051669865, 0.005037529, 0.004828512, 0.004908159, 0.004977275, 0.0056029283, 0.005233502, 0.0048554596, 0.0049089254, 0.0048818127, 0.0049084728, 0.004880431, 0.005376722, 0.0049555846, 0.0050253463, 0.0049334806, 0.0052300943, 0.0059857294, 0.004881787, 0.005383936, 0.0059584216, 0.005340414, 0.004828866, 0.005352909, 0.004881848, 0.0055725216, 0.004855603, 0.0053659654, 0.005136589, 0.0050097113, 0.0051662615, 0.0051504252, 0.005127237, 0.00537172, 0.004935218, 0.0053024674, 0.0058813724, 0.0054736002, 0.00488333, 0.0048826076, 0.0053627677, 0.0050851917, 0.0050390405, 0.004881647, 0.0052404003, 0.005344389, 0.0048283404, 0.004855441, 0.0048550325, 0.005206355, 0.0051959297, 0.00542016, 0.005092844, 0.005250171, 0.004881975, 0.0050103036, 0.0057840124, 0.0051425206, 0.0051087104, 0.0053901575, 0.0061913817, 0.0052906647, 0.0054577813, 0.0051934235, 0.0048017018, 0.00

2021-12-04 17:45:13,744 : DEBUG : updating topics
2021-12-04 17:45:13,839 : INFO : topic #51 (0.005): 0.000*"bar" + 0.000*"sampling" + 0.000*"population" + 0.000*"deviation" + 0.000*"z" + 0.000*"y" + 0.000*"dartmouth" + 0.000*"statistics" + 0.000*"study" + 0.000*"mu"
2021-12-04 17:45:13,840 : INFO : topic #77 (0.005): 0.000*"internet" + 0.000*"population" + 0.000*"bar" + 0.000*"z" + 0.000*"statistics" + 0.000*"ip" + 0.000*"sampling" + 0.000*"y" + 0.000*"cable" + 0.000*"cables"
2021-12-04 17:45:13,841 : INFO : topic #65 (0.006): 0.125*"node" + 0.115*"nodes" + 0.091*"subtree" + 0.065*"root" + 0.045*"recursive" + 0.027*"trees" + 0.020*"rooted" + 0.018*"leaf" + 0.015*"factorial" + 0.014*"recursion"
2021-12-04 17:45:13,841 : INFO : topic #73 (0.007): 0.140*"interface" + 0.055*"interfaces" + 0.048*"comparable" + 0.037*"implements" + 0.022*"ad" + 0.013*"abstract" + 0.013*"instances" + 0.012*"compiler" + 0.012*"adder" + 0.009*"greater"
2021-12-04 17:45:13,842 : INFO : topic #148 (0.007): 0.044

2021-12-04 17:45:14,604 : INFO : topic #88 (0.005): 0.000*"gcd" + 0.000*"cat" + 0.000*"greatest" + 0.000*"smallest" + 0.000*"inputs" + 0.000*"objectoriented" + 0.000*"brute" + 0.000*"complexity" + 0.000*"denominator" + 0.000*"scenario"
2021-12-04 17:45:14,605 : INFO : topic #65 (0.006): 0.125*"node" + 0.115*"nodes" + 0.091*"subtree" + 0.065*"root" + 0.044*"recursive" + 0.027*"trees" + 0.020*"rooted" + 0.018*"leaf" + 0.015*"factorial" + 0.014*"recursion"
2021-12-04 17:45:14,605 : INFO : topic #73 (0.007): 0.140*"interface" + 0.055*"interfaces" + 0.049*"comparable" + 0.037*"implements" + 0.022*"ad" + 0.013*"abstract" + 0.013*"instances" + 0.012*"compiler" + 0.012*"adder" + 0.009*"greater"
2021-12-04 17:45:14,605 : INFO : topic #148 (0.007): 0.044*"constructor" + 0.016*"private" + 0.015*"constructors" + 0.013*"rotate" + 0.011*"midterm" + 0.010*"literal" + 0.009*"default" + 0.009*"playground" + 0.009*"dimensions" + 0.009*"declared"
2021-12-04 17:45:14,608 : INFO : topic diff=0.120787, rho=

In [83]:
### after word2vec skimming
reduced = []
with open("reducedretrieval.dat", encoding='utf-8') as f:
    for line in f:
        reduced.append(line.split('.'))
with open("reducedanalytics.dat", encoding='utf-8') as f:
    for line in f:
        reduced.append(line.split('.'))
reduced = [[clean_sentence(s) for s in sentences] for sentences in reduced]
reduced = [[tokenize(s) for s in sentences] for sentences in reduced]

phrased = []
for sentences in reduced:
    ps = []
    for s in sentences:
        i = 0
        new_s = []
        while i < len(s) - 1:
            if (s[i+1] == p.get(s[i], False)):
                new_s.append(s[i] + "_" + s[i+1])
                i+=2
            else:
                new_s.append(s[i])
                i+=1
        nns =[]
        i = 0
        while i < len(new_s) - 1:
            if (new_s[i+1] == p.get(new_s[i], False)):
                nns.append(new_s[i] + "_" + new_s[i+1])
                i+=2
            else:
                nns.append(new_s[i])
                i+=1
        ps.append(nns)
    phrased.append(ps)
    
phrased_doc = []
for doc in phrased:
    d = []
    for s in doc:
        d+=s
    phrased_doc.append(d)
reduced_corpus = [dictionary.doc2bow(doc) for doc in phrased_doc]
phrased_doc

[['semantic',
  'consider',
  'word',
  'words',
  'help',
  'matching',
  'documents',
  'original_query',
  'word',
  'different',
  'lecture',
  'natural_language',
  'result',
  'robust',
  'general',
  'natural_language_processing',
  'techniques',
  'process',
  'lot',
  'finally',
  'going',
  'cover',
  'relation',
  'natural_language_processing',
  'technical',
  'code',
  'allow',
  'add',
  'additional',
  'words',
  'query',
  'additional',
  'words',
  'related',
  'means',
  'nlp_techniques',
  'general',
  'robust',
  'efficient',
  'implies',
  'today',
  'use',
  'fairly',
  'shallow',
  'nlp_techniques',
  'terms',
  'relations',
  'extracted',
  'current',
  'natural_language_processing',
  'example',
  'techniques',
  'allow',
  'extract',
  'entities',
  'relations',
  'mentioned',
  'matching',
  'query',
  'words',
  'text',
  'document',
  'chances',
  'document',
  'cause',
  'knowledge_graph',
  'contain',
  'entities',
  'relations',
  'goes',
  'simple',
  '

In [84]:
with open("results.dat", "w") as out:
    print(len(phrased_doc))
    for k in range(len(reduced)):
        top_topics = model.get_document_topics(reduced_corpus[k]) # [(topic_id, prob)]
        ####
        top_topics.sort(key=takelast, reverse=True)
        for topic in top_topics:
            print('topic {}'.format(k))
            topic_term_distribution = model.get_topic_terms(topic[0])
            for term in topic_term_distribution:
                print('{}, {}'.format(dictionary[term[0]], term[1]))
                out.write(dictionary[term[0]] + ",")
            print('prob: {}'.format(topic[-1]))
        out.write("\n")

96
topic 0
natural_language, 0.03314526006579399
chasing, 0.027551140636205673
dog, 0.027037665247917175
ambiguity, 0.02450171858072281
accuracy, 0.020269449800252914
bag, 0.01856209896504879
syntactic, 0.01855582371354103
phrases, 0.018526384606957436
nlp, 0.01656872034072876
john, 0.015468413010239601
prob: 0.9936357140541077
topic 1
pull_mode, 0.03491673991084099
push, 0.034916117787361145
querying, 0.03491494432091713
browsing, 0.03491493687033653
keywords, 0.027941545471549034
initiative, 0.027940694242715836
interest, 0.027939708903431892
mode, 0.020967010408639908
supported, 0.020965680480003357
browse, 0.02096503973007202
prob: 0.9865623116493225
topic 2
relevant_documents, 0.038744885474443436
assumptions, 0.021535446867346764
ranked_list, 0.017234129831194878
principle, 0.012933768332004547
r, 0.012932289391756058
approximation, 0.012931868433952332
strategies, 0.012931832112371922
sufficient, 0.012931685894727707
information_retrieval, 0.012931602075695992
q, 0.0129315992817

keywords, 0.027941545471549034
initiative, 0.027940694242715836
interest, 0.027939708903431892
mode, 0.020967010408639908
supported, 0.020965680480003357
browse, 0.02096503973007202
prob: 0.08033505082130432
topic 42
interface, 0.02176513895392418
listed, 0.01635950244963169
push, 0.014967366121709347
resources, 0.014868226833641529
studies, 0.014812709763646126
improving, 0.014524736441671848
feedback, 0.014524420723319054
strategies, 0.014524237252771854
indexing, 0.014523915946483612
engines, 0.014523723162710667
prob: 0.9863532781600952
topic 43
dog, 0.059457484632730484
pet, 0.04007481783628464
capital, 0.024560457095503807
flop, 0.024558888748288155
flip, 0.02197868935763836
morph, 0.020681900903582573
polymorphism, 0.01939159445464611
cast, 0.015515146777033806
choochoo, 0.0142257921397686
extends, 0.01422196626663208
prob: 0.7807257175445557
topic 43
real_world, 0.046600960195064545
nontext_data, 0.043283991515636444
sensor, 0.035828907042741776
actionable_knowledge, 0.02628528

base, 0.04943520203232765
leaf, 0.03369345888495445
recursion, 0.029552185907959938
dead, 0.013162042945623398
wrapper, 0.01153978519141674
leaves, 0.009586579166352749
iterative, 0.009582563303411007
prob: 0.8368647694587708
topic 84
precision_recall, 0.13948269188404083
precision, 0.09319084137678146
relevant_documents, 0.049105431884527206
system_b, 0.043087780475616455
f1, 0.037174124270677567
retrieved, 0.03439102694392204
arithmetic_mean, 0.030747421085834503
retrieve, 0.019749479368329048
tradeoff, 0.01973695680499077
ideal, 0.019725877791643143
prob: 0.1524275690317154
topic 85
opinion, 0.10086967796087265
opinions, 0.03076133131980896
holder, 0.019908251240849495
target, 0.01448179129511118
contrast, 0.014263068325817585
sentiment, 0.012672671116888523
trend, 0.012638828717172146
feeling, 0.01225509587675333
engines, 0.010864593088626862
expressed, 0.01042107492685318
prob: 0.994449257850647
topic 86
theta, 0.03896215185523033
sentiment, 0.022744281217455864
cluster, 0.0196881

In [10]:
def takelast(x):
    return x[-1]

In [13]:
from gensim.test.utils import datapath
# Save model to disk.
temp_file = datapath("model2.lda")
model.save(temp_file)

2021-11-27 22:20:07,084 : INFO : adding document #0 to Dictionary<0 unique tokens: []>
2021-11-27 22:20:07,084 : INFO : built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)
2021-11-27 22:20:07,085 : DEBUG : starting a new internal lifecycle event log for Dictionary
2021-11-27 22:20:07,085 : INFO : Dictionary lifecycle event {'msg': "built Dictionary<12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...> from 9 documents (total 29 corpus positions)", 'datetime': '2021-11-27T22:20:07.085097', 'gensim': '4.1.3.dev0', 'python': '3.8.8 (default, Apr 13 2021, 15:08:03) [MSC v.1916 64 bit (AMD64)]', 'platform': 'Windows-10-10.0.19041-SP0', 'event': 'created'}
2021-11-27 22:20:07,137 : DEBUG : starting a new internal lifecycle event log for LdaState
2021-11-27 22:20:07,137 : INFO : LdaState lifecycle event {'fname_or_handle': 'D:\\ProgramFiles\\conda\\lib\\site-packages\\gensim-4.1.3